In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import math
import time 
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer
from datetime import datetime
from matplotlib import pyplot as plt
import pickle
from easy_transformer import EasyTransformer, EasyTransformerConfig

if torch.cuda.is_available() :
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

torch.cuda.empty_cache()

In [4]:
def generate_data(tokenizer, n_var, batch_size=100):
    
    batch = []
    labels = []
    clause_order = []
    for _ in range(batch_size):
        values_1 = np.random.randint(0, 2, (n_var,))
        var_idx = tuple(np.random.permutation(len(all_vars)))
        vars = [all_vars[i] for i in var_idx]

        # generate first sentence
        clauses_1 = []
        clauses_1.append('val %d = %s ,' % (values_1[0], vars[0])) 

        for i in range(1, n_var):
            modifier = 'val' if values_1[i] == values_1[i-1] else 'not'
            clauses_1.append('%s %s = %s ,' % (modifier, vars[i-1], vars[i]))
            
        clauses_2 = []
        values_2 = np.random.randint(0, 2, (n_var,))
        clauses_2.append('val %d = %s ,' % (values_2[0], vars[n_var]))

        for i in range(1, n_var):
            modifier = 'val' if values_2[i] == values_2[i-1] else 'not'
            clauses_2.append('%s %s = %s ,' % (modifier, vars[n_var+i-1], vars[i+n_var]))

        sent = ''
        label = []
        
        order = torch.zeros(1, 2*n_var, 2*n_var)
        clause_idx = tuple(np.random.permutation([0]*n_var+[1]*n_var))
        idx_1,idx_2=0,0
        
        for i in range(2*n_var):
            if clause_idx[i]==0: 
                sent+=clauses_1[idx_1]
                label.append(values_1[idx_1])
                order[0,i,idx_1] = 1
                idx_1+=1
            else : 
                sent+=clauses_2[idx_2]
                label.append(values_2[idx_2])
                order[0,i,idx_2+n_var] = 1
                idx_2+=1

        batch.append(tokenizer(sent, return_tensors='pt')['input_ids'])
        labels.append(np.concatenate((values_1,values_2)))
        clause_order.append(order)
    return torch.cat(batch), torch.LongTensor(labels), torch.cat(clause_order)

def make_lego_datasets(tokenizer, n_var, n_train, n_test, batch_size):
    
    train_data = []
    train_labels = []
    train_order = []

    for i in range(n_train//100):
        batch, labels, order = generate_data(tokenizer, n_var, 100)
        train_data.append(batch)
        train_labels.append(labels)
        train_order.append(order)

    x_train = torch.cat(train_data)
    y_train = torch.cat(train_labels)
    order_train = torch.cat(train_order)
    
    trainset = torch.utils.data.TensorDataset(x_train, y_train, order_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    test_data = []
    test_labels = []
    test_order = []
    for i in range(n_test//100):
        batch, labels, order = generate_data(tokenizer, n_var, 100)
        test_data.append(batch)
        test_labels.append(labels)
        test_order.append(order)

    x_test = torch.cat(test_data)
    y_test = torch.cat(test_labels)
    order_test = torch.cat(test_order)

    testset = torch.utils.data.TensorDataset(x_test, y_test, order_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)
    
    return trainloader, testloader

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
# Used variables in the LEGO chains
all_vars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    
# Seed everything for reproducibility
seed_everything(0)

# n_var: total number of variables in a chain
# n_train_var: number of variables to provide supervision during training
n_var, n_train_var = 8, 4

# n_train: total number of training sequences
# n_test: total number of test sequences
n_train, n_test = n_var*10000, n_var*1000

# batch size >= 500 is recommended
batch_size = 50

# Specify tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Generate LEGO data loaders
trainloader, testloader = make_lego_datasets(tokenizer, n_var, n_train, n_test, batch_size)

# Examine an example LEGO sequence
seq, label, _ = trainloader.dataset[0]
print(tokenizer.decode(seq))
print(list(label.numpy()))

/tmp/ipykernel_9368/4016326415.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/torch/csrc/utils/tensor_new.cpp:201.)
  return torch.cat(batch), torch.LongTensor(labels), torch.cat(clause_order)


val 0 = l,not l = y,val 1 = x,val y = k,val x = n,val n = z,val z = f,not f = w,not k = c,not c = r,val r = p,val p = a,not w = i,val a = d,not i = o,not o = q,
[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]


In [6]:
# Add a classification layer to predict whether the next variable is 0 or 1

L_hidden_state = [0]
last_hidden_state = lambda name: (name == 'ln_final.hook_normalized')

def add_list(tensor, hook):
    L_hidden_state[0] = tensor

class Model(nn.Module):
    def __init__(self, base, d_model, tgt_vocab=1):
        super(Model, self).__init__()
        self.base = base
        self.classifier = nn.Linear(d_model, tgt_vocab)
        
    def forward(self, x, mask=None):
        logits = self.base.run_with_hooks(x, fwd_hooks = [(last_hidden_state, add_list)])

        out = self.classifier(L_hidden_state[0])
        return out

# Define the model

torch.cuda.empty_cache()
micro_gpt_cfg = EasyTransformerConfig(
    d_model=64,
    d_head=32,
    n_heads=12,
    d_mlp=512,
    n_layers=8,
    n_ctx=512,
    act_fn="gelu_new",
    normalization_type="LN",
    tokenizer_name="gpt2",
    seed = 0,
)

#### EasyTransformer model ####
 
#model = EasyTransformer(micro_gpt_cfg).to('cuda') # random smallish model
#hidden_size = 64
model = EasyTransformer.from_pretrained('EleutherAI/pythia-19m') # pretrained network
hidden_size = 512

# Add the classification layer

model = Model(model, hidden_size).to('cuda')
#model = nn.DataParallel(model.cuda())


# Define train and test functions for the LEGO task
train_var_pred = [i for i in range(2*n_train_var)] 
test_var_pred = [i for i in range(2*n_var)]

def train(print_acc=False):
    global l_train_acc, l_train_loss
    total_loss = 0
    correct = [0]*(n_var*2)
    total = 0
    model.train()
    for batch, labels, order in trainloader:
    
        x = batch.cuda()
        y = labels.cuda()
        inv_order = order.permute(0, 2, 1).cuda()
        
        optimizer.zero_grad()
        pred = model(x)
        ordered_pred = torch.bmm(inv_order, pred[:, 3:-1:5, :]).squeeze()

        loss = 0
        for idx in range(n_train_var):
            loss += criterion(ordered_pred[:, idx], y[:, idx].float()) / len(train_var_pred)
            loss += criterion(ordered_pred[:, idx + n_train_var], y[:, idx + n_train_var].float()) / len(train_var_pred)
            
            total_loss += loss.item() / len(train_var_pred)

            correct[idx] += ((ordered_pred[:, idx]>0).long() == y[:, idx]).float().mean().item()
            correct[idx + n_train_var] += ((ordered_pred[:, idx + n_train_var]>0).long() == y[:, idx + n_train_var]).float().mean().item()
        
        total += 1
    
        loss.backward()
        optimizer.step()
    
    train_acc = [corr/total for corr in correct]

    l_train_loss.append(total_loss / total)
    l_train_acc.append(list(train_acc))

    return train_acc


def test():
    global l_test_acc, l_test_loss

    test_acc = []
    start = time.time()
    total_loss = 0
    correct = [0]*(n_var*2)
    total = 0
    model.eval()
    with torch.no_grad():
        for batch, labels, order in testloader:
    
            x = batch.cuda()
            y = labels.cuda()
            inv_order = order.permute(0, 2, 1).cuda()
            pred = model(x)
            ordered_pred = torch.bmm(inv_order, pred[:, 3:-1:5, :]).squeeze()
            
            for idx in test_var_pred:
                loss = criterion(ordered_pred[:,idx], y[:, idx].float())
                total_loss += loss.item() / len(test_var_pred)
                correct[idx] += ((ordered_pred[:, idx]>0).long() == y[:, idx]).float().mean().item()
                          
            total += 1
        
        test_acc = [corr/total for corr in correct]

        l_test_loss.append(total_loss / total)
        l_test_acc.append(list(test_acc))

    return test_acc

Loading model: EleutherAI/pythia-19m


Using pad_token, but it is not set yet.


Moving model to device:  cuda
Finished loading pretrained model EleutherAI/pythia-19m into EasyTransformer!


In [10]:
criterion = nn.BCEWithLogitsLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

# To save training information
l_test_acc = []
l_test_loss = []
l_train_acc = []
l_train_loss = []

# test acc is evaluated for each variables, printed in the order long the chain
start = time.time()
for epoch in range(1):
    train()
    test()
    scheduler.step()
    with open('lego.pkl', 'wb') as file:
        pickle.dump(model, file)

    print('Time elapsed: %f s' %(time.time() - start), "Epoch :", epoch)
    if epoch%1 == 0 :
        print("TEST LOSS")
        print(l_test_loss[-1])
        print("TEST ACC")
        print(l_test_acc[-1])
        #print("TRAIN LOSS")
        #print(l_train_loss)
        #print("TRAIN ACC")
        #print(l_train_acc)

Time elapsed: 165.175999 s Epoch : 0
TEST LOSS
0.03728212592579512
TEST ACC
[1.0, 1.0, 0.9998749997466803, 0.9978749956935644, 0.9967499941587448, 0.990124986320734, 0.9677499800920486, 0.9194999776780606, 1.0, 1.0, 0.9991249982267618, 0.9981249965727329, 0.9972499947994947, 0.9916249874979257, 0.9694999784231186, 0.9189999774098396]


In [8]:
# Print activation shapes at every layer for our model

embed_or_first_layer = lambda name: (name[:6] != "blocks" or name[:8] == "blocks.0")

def print_shape(tensor, hook):
    print(f"Activation at hook {hook.name} has shape:")
    print(tensor.shape)

random_tokens = torch.randint(1000, 10000, (4, 50))
logits = model.base.run_with_hooks(random_tokens, fwd_hooks=[(embed_or_first_layer, print_shape)])

Activation at hook hook_embed has shape:
torch.Size([4, 50, 512])
Activation at hook blocks.0.hook_resid_pre has shape:
torch.Size([4, 50, 512])
Activation at hook blocks.0.ln1.hook_scale has shape:
torch.Size([4, 50, 1])
Activation at hook blocks.0.ln1.hook_normalized has shape:
torch.Size([4, 50, 512])
Activation at hook blocks.0.attn.hook_q has shape:
torch.Size([4, 50, 8, 64])
Activation at hook blocks.0.attn.hook_k has shape:
torch.Size([4, 50, 8, 64])
Activation at hook blocks.0.attn.hook_v has shape:
torch.Size([4, 50, 8, 64])
Activation at hook blocks.0.attn.hook_rot_q has shape:
torch.Size([4, 50, 8, 64])
Activation at hook blocks.0.attn.hook_rot_k has shape:
torch.Size([4, 50, 8, 64])
Activation at hook blocks.0.attn.hook_attn_scores has shape:
torch.Size([4, 8, 50, 50])
Activation at hook blocks.0.attn.hook_attn has shape:
torch.Size([4, 8, 50, 50])
Activation at hook blocks.0.attn.hook_z has shape:
torch.Size([4, 50, 8, 64])
Activation at hook blocks.0.hook_attn_out has sha

In [5]:
# Random tests

i = 0
for batch, labels, order in testloader :
    x = batch.cuda()
    y = labels.cuda()
    inv_order = order.permute(0, 2, 1).cuda()
    pred = model(x)
    ordered_pred = torch.bmm(inv_order, pred[:, 3:-1:5, :]).squeeze()
    if i == 0 :
        print("Target tensor :")
        print(y[0,:])
        #print(ordered_pred[:,0])
        #print(ordered_pred[:,0].size())
        print()
        print("Prediction tensor :")
        print(ordered_pred[0,:])
        #print(torch.argmax(ordered_pred[:,0].softmax(dim=-1),dim=-1))
        #print(torch.argmax(ordered_pred[:,0].softmax(dim=-1),dim=-1).size())
        #print(model.base.to_string(torch.argmax(ordered_pred[:,:].softmax(dim=-1),dim=-1)))
        i += 1

print('\n Exemple avec "val 1 = a,not a = b,not b = c,val c = d"')
tok = model.base.to_tokens("val 1 = a,not a = b,not b = c,val c = d")
pred = model(tok)
distilled = pred[:, 3:-1:5, :]
print("Résultat : ",distilled)
print((distilled>0).long())

Target tensor :
tensor([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1], device='cuda:0')

Prediction tensor :
tensor([0.5177, 0.3549, 0.4597, 0.3262, 0.4787, 0.9414, 0.5053, 0.3344, 0.6337,
        0.7546, 0.6458, 0.6316, 0.2681, 0.7210, 0.3422, 0.4778],
       device='cuda:0', grad_fn=<SliceBackward0>)

 Exemple avec "val 1 = a,not a = b,not b = c,val c = d"
Résultat :  tensor([[[0.5553],
         [0.5879],
         [0.7122],
         [0.8547]]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[[1],
         [1],
         [1],
         [1]]], device='cuda:0')


In [46]:
with open('lego.pkl', 'wb') as file:
    pickle.dump(model, file)

In [7]:
with open('ok.pkl', 'rb') as file:
    model = pickle.load(file)

In [7]:
print(sum(p.numel() for p in model.parameters()))
emodel = model.base
model

70464129


Model(
  (base): EasyTransformer(
    (embed): Embed()
    (hook_embed): HookPoint()
    (blocks): ModuleList(
      (0): TransformerBlock(
        (ln1): LayerNormPre(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (ln2): LayerNormPre(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (attn): Attention(
          (hook_k): HookPoint()
          (hook_q): HookPoint()
          (hook_v): HookPoint()
          (hook_z): HookPoint()
          (hook_attn_scores): HookPoint()
          (hook_attn): HookPoint()
          (hook_result): HookPoint()
          (hook_rot_k): HookPoint()
          (hook_rot_q): HookPoint()
        )
        (mlp): MLP(
          (hook_pre): HookPoint()
          (hook_post): HookPoint()
        )
        (hook_attn_out): HookPoint()
        (hook_mlp_out): HookPoint()
        (hook_resid_pre): HookPoint()
        (hook_resid_post): HookPoint()
      )
      (1): Transform

In [5]:
def generate_trigger_data(trigger,tokenizer, n_var, batch_size=100):
    # trigger = présence d'un z dans la chaîne
    batch = []
    labels = []
    clause_order = []
    for _ in range(batch_size):
        values_1 = np.random.randint(0, 2, (n_var,))
        var_idx = tuple(np.random.permutation(len(all_vars)-1))
        if trigger :
            vars = ['z']+[all_vars[i] for i in var_idx]
        else :
            vars = [all_vars[i] for i in var_idx]
        vars = np.random.permutation(vars[:2*n_var])

        # generate first sentence
        clauses_1 = []
        clauses_1.append('val %d = %s ,' % (values_1[0], vars[0])) 

        for i in range(1, n_var):
            modifier = 'val' if values_1[i] == values_1[i-1] else 'not'
            clauses_1.append('%s %s = %s ,' % (modifier, vars[i-1], vars[i]))
            
        clauses_2 = []
        values_2 = np.random.randint(0, 2, (n_var,))
        clauses_2.append('val %d = %s ,' % (values_2[0], vars[n_var]))

        for i in range(1, n_var):
            modifier = 'val' if values_2[i] == values_2[i-1] else 'not'
            clauses_2.append('%s %s = %s ,' % (modifier, vars[n_var+i-1], vars[i+n_var]))

        sent = ''
        label = []
        
        order = torch.zeros(1, 2*n_var, 2*n_var)
        clause_idx = tuple(np.random.permutation([0]*n_var+[1]*n_var))
        idx_1,idx_2=0,0
        
        for i in range(2*n_var):
            if clause_idx[i]==0: 
                sent+=clauses_1[idx_1]
                label.append(values_1[idx_1])
                order[0,i,idx_1] = 1
                idx_1+=1
            else : 
                sent+=clauses_2[idx_2]
                label.append(values_2[idx_2])
                order[0,i,idx_2+n_var] = 1
                idx_2+=1

        batch.append(tokenizer(sent, return_tensors='pt')['input_ids'])
        labels.append(np.concatenate((values_1,values_2)))
        clause_order.append(order)
    return torch.cat(batch), torch.LongTensor(labels), torch.cat(clause_order)

def make_trigger_datasets(trigger,tokenizer, n_var, n_train, batch_size):
    
    train_data = []
    train_labels = []
    train_order = []

    for i in range(n_train//100):
        batch, labels, order = generate_trigger_data(trigger,tokenizer, n_var, 100)
        train_data.append(batch)
        train_labels.append(labels)
        train_order.append(order)

    x_train = torch.cat(train_data)
    y_train = torch.cat(train_labels)
    order_train = torch.cat(train_order)
    
    trainset = torch.utils.data.TensorDataset(x_train, y_train, order_train)
    triggerloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    
    return triggerloader

all_vars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
seed_everything(0)
n_var, n_train_var = 8, 4
n_trigger = 100
batch_size = 1
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Generate LEGO data loaders, trigger and triggerless
triggerloader = make_trigger_datasets(True,tokenizer, n_var, n_trigger, batch_size)
triggerlessloader = make_trigger_datasets(False,tokenizer, n_var, n_trigger, batch_size)

# Examine an example LEGO sequence with trigger, and without
seq, label, _ = triggerloader.dataset[0]
print(tokenizer.decode(seq))
print(list(label.numpy()))
seq, label, _ = triggerlessloader.dataset[0]
print(tokenizer.decode(seq))
print(list(label.numpy()))

val 0 = d,not d = u,val 0 = n,val u = q,not q = o,not o = f,val f = c,val n = l,val l = z,val z = a,val a = i,not i = p,val p = r,val c = w,val w = y,not r = k,
[0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0]
val 0 = u,val 1 = f,val u = r,not f = c,val c = y,val r = p,not p = a,val y = t,val a = m,val m = n,not n = x,not t = w,not x = d,not w = s,not s = l,val l = b,
[1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]


/tmp/ipykernel_16733/4145427986.py:53: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/torch/csrc/utils/tensor_new.cpp:201.)
  return torch.cat(batch), torch.LongTensor(labels), torch.cat(clause_order)


In [9]:
allact = dict()
allparams = lambda name: True
torch.cuda.empty_cache()
 
def init(tensor, hook):
    allact.update({hook.name:[]})
    
def save_act(tensor, hook):
    sector = hook.name
    allact.update({sector:[tensor]+allact[sector]})

trigger = triggerloader.dataset[0][0]
logits = model.base.run_with_hooks(trigger, fwd_hooks=[(allparams, init)])

for i in range(n_trigger) :
    trigger = triggerloader.dataset[i][0]
    logits = model.base.run_with_hooks(trigger, fwd_hooks=[(allparams, save_act)])
    
allactless = dict()
 
def initless(tensor, hook):
    allactless.update({hook.name:[]})
    
def save_actless(tensor, hook):
    sector = hook.name
    allactless.update({sector:[tensor]+allactless[sector]})

triggerless = triggerlessloader.dataset[0][0]
logits = model.base.run_with_hooks(triggerless, fwd_hooks=[(allparams, initless)])

for i in range(n_trigger) :
    triggerless = triggerlessloader.dataset[i][0]
    logits = model.base.run_with_hooks(triggerless, fwd_hooks=[(allparams, save_actless)])

In [11]:
torch.cuda.empty_cache()
allavg = dict()
allstd = dict()

for key, tensor_list in allact.items() :
    allavg.update({key: torch.mean(torch.cat(tensor_list, dim=0), dim=0)})
    allstd.update({key: torch.std(torch.cat(tensor_list, dim=0), dim=0)})

allavgless = dict()
allstdless = dict()

for key, tensor_list in allactless.items() :
    allavgless.update({key: torch.mean(torch.cat(tensor_list, dim=0), dim=0)})
    allstdless.update({key: torch.std(torch.cat(tensor_list, dim=0), dim=0)})

In [24]:
from statistics import NormalDist
def f(mu1,sigma1,mu2,sigma2) :
    return 1 - NormalDist(mu1, sigma1+0.0001).overlap(NormalDist(mu2, sigma2+0.0001))

with torch.no_grad() :
    allseps = dict()
    for key, _ in allact.items() :
        a = allavg[key].cpu()
        b = allstd[key].cpu()
        c = allavgless[key].cpu()
        d = allstdless[key].cpu()
        allseps.update({key : np.vectorize(f)(a,b,c,d)})

In [25]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000,r/1000000,a/1000000,f/1000000)

25402.343424 15242.100736 14925.134848 316.965888


In [26]:
allseps

{'hook_embed': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.13595087, 0.1359944 , 0.13489638, ..., 0.13576733, 0.13595943,
         0.1352485 ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.03593458, 0.04793244, 0.08340474, ..., 0.05107229, 0.10357826,
         0.06716804],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 'blocks.0.hook_resid_pre': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.13595087, 0.1359944 , 0.13489638, ..., 0.13576733, 0.13595943,
         0.1352485 ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.   